In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('../data/processed/cleaned_dataset.csv')
df.columns

Index(['S_NAME', 'QUESTION', 'ANSWER', 'OPEN_TIME_', 'RESOLVE_TIME_',
       'CLOSE_TIME_', 'ATC_NEXT_BREACH_', 'TITLE', 'SUBCATEGORY', 'REG',
       'PRIORITY', 'AVARIYA', 'SOURCE', 'QUESTION_FULL', 'SUBCATEGORY_CLEAN',
       'hour_sin', 'hour_cos', 'day_of_week_sin', 'day_of_week_cos',
       'is_weekend', 'day_of_month_sin', 'day_of_month_cos'],
      dtype='object')

In [4]:
titles_df = df[['TITLE', 'S_NAME', 'SUBCATEGORY_CLEAN']]
titles_df.sample(3)

,TITLE,S_NAME,SUBCATEGORY_CLEAN
34089,блокировка уз,СЭД,Доступ к ИСОД
373316,блокировка уз,СЭД,Доступ к ИСОД
696448,"техническая ошибка ""миграционный учет""",ЕИР РМУ,Прочее


In [47]:
import re

def clean_title(t: str) -> str:
    t = t.strip()
    t = re.sub(r'\s+', ' ', t)  # лишние пробелы
    t = re.sub(r'[\"\'<>]', '', t)  # кавычки, скобки
    t = re.sub(r'сц\s*\d*', '', t)  # удаляем "сц" и любые следующие цифры
    t = re.sub(r'\(\s*\)', '', t)  # удаляем пустые скобки
    # return t[:100]  # обрезка до 100 символов
    return t

titles_df['title_clean'] = titles_df['TITLE'].apply(clean_title)

C:\Users\mew\AppData\Local\Temp\ipykernel_19176\2243529552.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  titles_df['title_clean'] = titles_df['TITLE'].apply(clean_title)


In [48]:
titles_df[titles_df['title_clean'].str.contains("сц")]['title_clean']

Series([], Name: title_clean, dtype: object)

In [50]:
from rapidfuzz import process, fuzz
import json

SIMILARITY_SCORE = 97

# 1. Ищем похожие классы по Левенштейну
unique_classes = titles_df['title_clean'].unique()
similar_map = {}

# Аггрегируем классы со схожестью > 90
for cls in unique_classes:
    matches = process.extract(
        cls, 
        unique_classes, 
        scorer=fuzz.ratio, 
        limit=None
    )
    similar_map[cls] = [m[0] for m in matches if m[0] != cls and m[1] >= SIMILARITY_SCORE]

In [51]:
print("\nПохожие классы (порог 90%):")
for i, (k, v) in enumerate(similar_map.items()):
    if i > 100:
        break
    if v:
        print(f"- {k} -> {v}")


Похожие классы (порог 90%):
- запрос информации по фис гибдд-м -> ['+запрос информации по фис гибдд-м', '0запрос информации по фис гибдд-м', 'юзапрос информации по фис гибдд-м', ',запрос информации по фис гибдд-м']
- корректировка данных в сооп -> ['корректировка данных в сооп\\', 'корректировка даннеых в сооп', 'корректировка данныхх в сооп', 'ёкорректировка данных в сооп', 'корректировка данны в сооп', 'корректировк данных в сооп', 'корректировка данныхв сооп', 'крректировка данных в сооп', 'коррекировка данных в сооп', 'корректировка даных в сооп', 'корректировка данных в соо', 'корректровка данных в сооп']
- запрос информации по сооп -> ['зпрос информации по сооп']
- запрос информации -> ['запрос информации ']
- запрос информации по содч -> ['пзапрос информации по содч', '.запрос информации по содч']
- корректировка прав в сэд -> ['dкорректировка прав в сэд']
- запрос информации по ибд-м -> ['.запрос информации по ибд-м']
- сэд. приволжский федеральный округ. снижение производител

In [53]:
PREFERRED_SUBCATS = []

parent = {c: c for c in unique_classes}

def find(x):
    while parent[x] != x:
        parent[x] = parent[parent[x]]
        x = parent[x]
    return x

def union(a, b):
    ra, rb = find(a), find(b)
    if ra != rb:
        parent[rb] = ra

for cls, similars in similar_map.items():
    for s in similars:
        union(cls, s)

clusters = {}
for c in unique_classes:
    root = find(c)
    clusters.setdefault(root, []).append(c)

# Выбор референсов
counts = titles_df['title_clean'].value_counts()
similar_mapping = {}

for root, members in clusters.items():
    # Проверка на user-preferred
    preferred = None
    for pref in PREFERRED_SUBCATS:
        if pref in members:
            preferred = pref
            break
    
    if preferred:
        rep = preferred
    else:
        # Самый частый класс в кластере
        rep = max(members, key=lambda x: counts.get(x, 0))
    
    for m in members:
        similar_mapping[m] = rep

In [ ]:
df['TITLE'].sample(10)

295535     добавление в список рассылки
114563       запрос информации по судис
74123          восстановление пароля уз
54313            разблокировка уз в ошс
335741       запрос информации по ибд-м
96217          корректировка прав в сэд
105204         восстановление пароля уз
406184       запрос информации по судис
162273    предоставление доступа к сооп
627638              изменение данных уз
Name: TITLE, dtype: object

: 

In [81]:
arr = np.load('../data/features/title_bge_m3.npz')["ids"]

RangeIndex(start=0, stop=699869, step=1)

In [68]:
title_vecs = np.load('../data/features/title_bge_m3.npz')["features"]

In [79]:
title_vecs[484204]

array([-0.0119501 ,  0.02293888, -0.07833769, ..., -0.05120265,
        0.05019483,  0.02272015], dtype=float32)